In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

import sys
try:
    import cartopy.crs as ccrs
except ImportError:
    pass

if 'cartopy.crs' not in sys.modules:
    get_cartopy = False
    print('You have not imported the {} module'.format('cartopy.crs'))
else:
    get_cartopy = True

In [ ]:
with open("observations", "r") as f:
    obs_df = pd.read_csv(f, index_col=0)
f.close()

with open("outliers", "r") as f:
    outliers = pd.read_csv(f)
f.close()

my_outliers = outliers[outliers.time<=np.min(outliers.time)]

In [ ]:
projection = ccrs.PlateCarree(central_longitude=0)
fig = plt.figure(figsize=(10,10))

# Draw island
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.coastlines()

# Produce gridlines, coordinate labels
ax.set_extent([-45, 40, -30, 10], ccrs.PlateCarree())
ax.gridlines(draw_labels=True, crs=projection)

# Add points along flight path and color for altitude
measurements = ax.scatter(
    my_outliers[(my_outliers.missing==False) & (my_outliers.outlier==False)].longitude,
    my_outliers[(my_outliers.missing==False) & (my_outliers.outlier==False)].latitude,
    zorder=1, alpha=1,
    c=my_outliers[(my_outliers.missing==False) & (my_outliers.outlier==False)].meanResponse, cmap="Blues"
)

cbar = plt.colorbar(measurements, orientation = 'horizontal')
cbar.set_label('Measured AOD', fontsize = 14)

ax.scatter(
    my_outliers[my_outliers.missing==True].longitude,
    my_outliers[my_outliers.missing==True].latitude,
    zorder=1, alpha=0.5,
    c="r", marker="s", s=50,
    label='Missing measurement'
)
ax.scatter(
    my_outliers[my_outliers.outlier==True].longitude,
    my_outliers[my_outliers.outlier==True].latitude,
    zorder=1, alpha=0.5,
    c="g", marker="s", s=50,
    label='Outlier'
)

plt.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.1), ncol = 2, fontsize = 14)
plt.show()